In [ ]:
%pip install --upgrade --quiet \
    "google-cloud-aiplatform[agent_engines,langchain]" \
    cloudpickle==3.0.0 \
    "pydantic>=2.10" \
    requests

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
PROJECT_ID = "my-project-agent-32919"
#LOCATION = "us-east1"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://sree-agent-deploy-bucket" 

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

In [ ]:
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import LangchainAgent

In [ ]:

model = "gemini-2.0-flash"

In [ ]:
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

In [ ]:
get_exchange_rate(currency_from="USD", currency_to="SEK")

In [ ]:
agent = LangchainAgent(
    model=model,
    tools=[get_exchange_rate],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [10]:
agent.query(input="What's the exchange rate from US dollars to Swedish currency today?")

Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised FailedPrecondition: 400 Project `430369260773` is not allowed to use Publisher Model `projects/my-project-agent-32919/locations/us-central1/publishers/google/models/gemini-1.5-pro`.
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised FailedPrecondition: 400 Project `430369260773` is not allowed to use Publisher Model `projects/my-project-agent-32919/locations/us-central1/publishers/google/models/gemini-1.5-pro`.
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised FailedPrecondition: 400 Project `430369260773` is not allowed to use Publisher Model `projects/my-project-agent-32919/locations/us-central1/publishers/google/models/gemini-1.5-pro`.
Retrying langchain_google_vertexai.chat_models

FailedPrecondition: 400 Project `430369260773` is not allowed to use Publisher Model `projects/my-project-agent-32919/locations/us-central1/publishers/google/models/gemini-1.5-pro`